In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the Dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load transactions from pandas.
df = pd.read_csv("/kaggle/input/beer-and-diaper/beer and diaper.csv", header=None)

# Print the header
print(df.head())

In [ ]:
# Split transaction strings into lists.
transactions = df.iloc[:,1].apply(lambda t: t.split(','))

# Convert DataFrame into list of strings.
transactions = list(transactions)

# Print the second transaction
print(transactions[1])

# Count the number of transactions that contain bread and milk
transactions.count(['bread', 'milk'])

In [ ]:
#df['Transactions']= df.values.tolist()
#df

In [ ]:
#df['Transactions'] = df['Transactions'].apply(lambda x: [i for i in x if str(i) != "nan"])

In [ ]:
#df

In [ ]:
#transactions = df['Transactions']

In [ ]:
# Convert DataFrame column into list of strings
#transactions = list(transactions)

In [ ]:
#transactions.count(['burgers', 'meatballs', 'eggs'])

In [ ]:
from itertools import permutations

# Extract unique items.
flattened = [item for transaction in transactions for item in transaction]
items = list(set(flattened))

# Compute and print rules.
rules = list(permutations(items, 2))
print(rules)

In [ ]:
# Print the number of rules
print(len(rules))

## Simplest Metric

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Instantiate transaction encoder
encoder = TransactionEncoder().fit(transactions)

In [ ]:
# One-hot encode itemsets by applying fit and transform
onehot = encoder.transform(transactions)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)
print(onehot)

In [ ]:
# Computing Support for Single Items
print(onehot.mean())

In [ ]:
# Computing Support for Multiple Items
import numpy as np

# Define itemset that contains almonds and zucchini
onehot['beer+bread'] = np.logical_and(onehot['beer'], onehot['bread'])

print(onehot.mean())

In [ ]:
onehot=onehot.drop('beer+bread', axis=1)
onehot

## Condifence & Lift

In [ ]:
# Print first five items
print(onehot.head())

In [ ]:
# Computing support.
supportBD = np.logical_and(onehot['beer'],onehot['diaper']).mean()
supportB = onehot['beer'].mean()
supportD = onehot['diaper'].mean()

# Compute and print confidence and lift.
confidence = supportBD / supportB
lift = supportBD / (supportB * supportD)

# Print results.
print(supportD, confidence, lift)

## Computing Leverage

In [ ]:
# Compute support for "asparagus" and "almonds"
supportBD = np.logical_and(onehot['beer'], onehot['diaper']).mean()

# Compute support for "asparagus"
supportB = onehot['beer'].mean()

# Compute support for "almonds"
supportD = onehot['diaper'].mean()

# Compute and print leverage
leverage = supportBD - supportB * supportD
print(leverage)

## Computing Conviction

In [ ]:
# Compute support for "asparagus" and "almonds"
supportBD = np.logical_and(onehot['beer'], onehot['diaper']).mean()

# Compute support for "asparagus"
supportB = onehot['beer'].mean()

# Compute support for NOT "almonds"
supportnD = 1.0 - onehot['diaper'].mean()

# Compute support for "asparagus" and NOT "almonds"
supportBnD = supportB - supportBD

# Compute conviction
conviction = supportB*supportnD / supportBnD
print(conviction)

## Calculating Metrics

In [ ]:
# Let's define the functions to calculate the metrics from the original data.
from itertools import permutations

def supportA(itemA, df):
    return float(df[itemA].mean())    

def supportB(itemB, df):
    return float(df[itemB].mean())

def confidence(itemA,itemB,df):
    return float(np.logical_and(df[itemA],df[itemB]).mean() /(df[itemA].mean()))

def lift(itemA,itemB,df):
    return float(np.logical_and(df[itemA],df[itemB]).mean() /(df[itemA].mean() * df[itemB].mean()))

def leverage(itemA,itemB,df):
    return np.logical_and(df[itemA],df[itemB]).mean() - (df[itemA].mean()*df[itemB].mean())

def conviction(itemA, itemB, df):
    # Compute support for A and B
    supportAB = np.logical_and(df[itemA], df[itemB]).mean()
    # Compute support for A
    supportA = df[itemA].mean()
    # Compute support for not B
    supportnB = 1.0 - df[itemB].mean()
    # Compute support for A not B
    supportAnB = supportA - supportAB
    # Compute conviction
    return float(supportA*supportnB / supportAnB)

In [ ]:
item_pairs = list()
for itemA,itemB in permutations(onehot,2):
    item_pairs.append(list((itemA,itemB, #names
                            onehot[itemA].sum(),onehot[itemB].sum(), #individual count
                            np.logical_and(onehot[itemA],onehot[itemB]).sum(), #pair count
                            supportA(itemA, onehot),
                            supportB(itemB, onehot),
                            confidence(itemA,itemB,onehot), #confidence
                            lift(itemA,itemB,onehot), #lift
                            leverage(itemA,itemB,onehot), # leverage
                            conviction(itemA, itemB, onehot)
                            ))) # 

In [ ]:
item_pairs = pd.DataFrame(item_pairs,columns = ['itemA','itemB',
                                                'countItemA','countItemB',
                                                'countItemA&B',
                                                'Antecedent Support',
                                                'Consequent Support',
                                                'Confidence',
                                                'Lift',
                                                'Leverage',
                                                'Conviction'])

item_pairs.sample(5)

## Performing Multi-Metric Filtering

In [ ]:
# Select subset of rules with low consequent support.
rules = item_pairs[item_pairs['Consequent Support'] < 0.05]
print(len(rules))

In [ ]:
rules

In [ ]:
# Select subset of rules with lift > 1.5.
rules_2 = rules[rules['Lift'] > 1.5]
print(len(rules_2))

In [ ]:
rules_2

## Apriori Algorithm

In [ ]:
# Import Apriori algorithm
from mlxtend.frequent_patterns import apriori

In [ ]:
# Compute frequent itemsets
frequent_itemsets = apriori(onehot, min_support = 0.0005,max_len = 4, use_colnames = True)

# Print number of itemsets
print(len(frequent_itemsets))

In [ ]:
# Print frequent itemsets
print(frequent_itemsets.head())

## Apriori and Computing Association Rule 

In [ ]:
# Import Apriori algorithm
from mlxtend.frequent_patterns import apriori, association_rules

# Compute association rules
Arules = association_rules(frequent_itemsets,
                           metric = "support",
                           min_threshold = 0.005)

In [ ]:
Arules

In [ ]:
# Print the rules.
print(Arules)

In [ ]:
# Raise the threshold
# Compute association rules
Arules_2 = association_rules(frequent_itemsets,
                           metric = "support",
                           min_threshold = 0.010)

# Print the rules.
print(Arules_2)

In [ ]:
# Raise the threshold
# Compute association rules
Arules_3 = association_rules(frequent_itemsets,
                           metric = "support",
                           min_threshold = 0.050)

# Print the rules.
print(Arules_3)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(10,5))
sns.boxplot(x='antecedent support', data=Arules, linewidth=0.9, color="royalblue", ax=ax[0])
sns.boxplot(x='support', data=Arules, linewidth=0.9, color="royalblue", ax=ax[1])
sns.boxplot(x='confidence', data=Arules, linewidth=0.9, color="royalblue", ax=ax[2])
plt.tight_layout()
plt.show()

In [ ]:
filtered_rules = Arules[(Arules['antecedent support'] > 0.5) &
                        (Arules['support'] > 0.3) &
                        (Arules['confidence'] > 0.5) &
                        (Arules['lift'] > 1.00)]

In [ ]:
filtered_rules

### "Beer and Diaper" is the most highly associated items.